# 第5章: 係り受け解析
夏目漱石の小説『吾輩は猫である』の文章（neko.txt）をCaboChaを使って係り受け解析し，その結果をneko.txt.cabochaというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

In [24]:
import CaboCha

def parse():
    parser = CaboCha.Parser()
    with open('neko.txt') as input, open('neko.txt.cabocha', 'w') as f:
        for line in input:
            f.write(parser.parse(line).toString(CaboCha.FORMAT_LATTICE))

parse()

## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文をMorphオブジェクトのリストとして表現し，3文目の形態素列を表示せよ．

In [25]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
    
    def __str__(self):
        return 'surface: {} base: {}, pos: {}, pos1: {}'.format(self.surface, self.base, self.pos, self.pos1)
    
    def is_symbol(self):
        return self.pos == '記号'
    
    def is_noun(self):
        return self.pos == '名詞'
    
    def is_verb(self):
        return self.pos == '動詞'

    def is_joshi(self):
        return self.pos == '助詞'
    
    def is_sahen_noun(self):
        return self.is_noun() and self.pos1 == 'サ変接続'

In [26]:
def cabocha_morph_reader():
    morphs = []
    with open('neko.txt.cabocha') as f:
        for line in f:
            line = line.rstrip()
            if line == 'EOS':
                yield morphs
                morphs = []
                continue
            if line.startswith('*'):
                continue
            surface, others = line.split('\t')
            others = others.split(',')
            morphs.append(Morph(surface,  others[6], others[0], others[1]))
        raise StopIteration

for i, morphs in enumerate(cabocha_morph_reader(), 1):
    if i == 3:
        for morph in morphs:
            print(morph)
        break

surface: 　 base: 　, pos: 記号, pos1: 空白
surface: 吾輩 base: 吾輩, pos: 名詞, pos1: 代名詞
surface: は base: は, pos: 助詞, pos1: 係助詞
surface: 猫 base: 猫, pos: 名詞, pos1: 一般
surface: で base: だ, pos: 助動詞, pos1: *
surface: ある base: ある, pos: 助動詞, pos1: *
surface: 。 base: 。, pos: 記号, pos1: 句点


## 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．

In [27]:
class Chunk:
    def __init__(self):
        self.morphs = []
        self.dst = -1
        self.srcs = []
    
    def __str__(self):
        return 'morphs: {} dst: {} srcs: {}'.format(self.surface(), self.dst, self.srcs)
    
    def surface(self):
        ret = ''
        for morph in self.morphs:
            if morph.is_symbol():
                continue
            ret += morph.surface
        return ret
    
    def is_include_noun(self):
        return any([morph.is_noun() for morph in self.morphs])
    
    def is_include_verb(self):
        return any([morph.is_verb() for morph in self.morphs])

    def is_include_joshi(self):
        return any([morph.is_joshi() for morph in self.morphs])
    
    def is_include_sahen_wo(self):
        return len(self.sahen_wo()) > 0

    def predicate(self):
        # 最初の動詞
        for morph in self.morphs:
            if morph.is_verb():
                return morph.base
        else:
            return ''
    
    def joshi(self):
        # 最後の助詞
        for morph in self.morphs[::-1]:
            if morph.is_joshi():
                return morph.surface
        else:
            return ''
    
    def sahen_wo(self):
        morphs_size = len(self.morphs)
        for i, morph in enumerate(self.morphs):
            if i < morphs_size-1 and morph.is_sahen_noun() and self.morphs[i+1].surface == 'を':
                return morph.surface + self.morphs[i+1].surface
        else:
            return ''
    
    def masked_noun_phrase(self, val='X'):
        ret = []
        for morph in self.morphs:
            if morph.is_noun():
                ret.append(val)
            else:
                ret.append(morph.surface)
        return ''.join(ret)
    
    def masked_noun(self, val='X'):
        for morph in self.morphs:
            if morph.is_noun():
                return val
        else:
            return ''

In [28]:
class ChunkStore:
    def __init__(self):
        self.chunks = {}

    def update_chunk(self, id, **args):
        id = int(id)
        if id in self.chunks:
            pass
        else:
            self.chunks[id] = Chunk()

        if 'dst' in args:
            self.chunks[id].dst = int(args['dst'])
        if 'morph' in args:
            self.chunks[id].morphs.append(args['morph'])
        if 'src' in args:
            self.chunks[id].srcs.append(int(args['src']))
    
    def get_chunks(self):
        return list(self.chunks.values())

def cabocha_reader():
    current_chunk_id = None
    chunk_store = ChunkStore()
    with open('neko.txt.cabocha') as f:
        for line in f:
            line = line.rstrip()
            if line == 'EOS':
                yield chunk_store.get_chunks()
                chunk_store = ChunkStore()
                continue
            if line.startswith('*'): # Chunk
                column = line.split(' ')
                current_chunk_id = column[1]
                dst = int(column[2][:-1]) # remove 'D'
                chunk_store.update_chunk(current_chunk_id, dst=dst)
                if dst > -1:
                    chunk_store.update_chunk(dst, src=current_chunk_id)
            else: # Morph
                surface, others = line.split('\t')
                others = others.split(',')
                chunk_store.update_chunk(current_chunk_id, morph=Morph(surface,  others[6], others[0], others[1]))
        raise StopIteration


for i, chunks in enumerate(cabocha_reader(), 1):
    if i == 8:
        for j, chunk in enumerate(chunks):
            print(j, chunk)
        break
            

0 morphs: 吾輩は dst: 5 srcs: []
1 morphs: ここで dst: 2 srcs: []
2 morphs: 始めて dst: 3 srcs: [1]
3 morphs: 人間という dst: 4 srcs: [2]
4 morphs: ものを dst: 5 srcs: [3]
5 morphs: 見た dst: -1 srcs: [0, 4]


## 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [29]:
# 最初の10個を表示
total = 0
for i, chunks in enumerate(cabocha_reader(), 1):
    if total >= 10:
        break
    for j, chunk in enumerate(chunks):
        if total >= 10:
            break
        if chunk.dst > -1:
            surface = chunk.surface()
            dist_surface = chunks[chunk.dst].surface()
            if surface == '' or dist_surface == '':
                continue
            print('{}\t{}'.format(surface, dist_surface))
            total += 1

吾輩は	猫である
名前は	無い
まだ	無い
どこで	生れたか
生れたか	つかぬ
とんと	つかぬ
見当が	つかぬ
何でも	薄暗い
薄暗い	所で
じめじめした	所で


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [30]:
# 最初の10個を表示
total = 0
for i, chunks in enumerate(cabocha_reader(), 1):
    if total >= 10:
        break
    for j, chunk in enumerate(chunks):
        if total >= 10:
            break
        if chunk.dst > -1:
            if chunk.is_include_noun() and chunks[chunk.dst].is_include_verb():
                surface = chunk.surface()
                dist_surface = chunks[chunk.dst].surface()
                if surface == '' or dist_surface == '':
                    continue
                print('{}\t{}'.format(surface, dist_surface))
                total += 1

どこで	生れたか
見当が	つかぬ
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している
吾輩は	見た
ここで	始めて
ものを	見た
あとで	聞くと
我々を	捕えて


## 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

pass

## 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

* 動詞を含む文節において，最左の動詞の基本形を述語とする
* 述語に係る助詞を格とする
* 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる

「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

```
始める  で
見る    は を
```
このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

* コーパス中で頻出する述語と格パターンの組み合わせ
* 「する」「見る」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

In [31]:
with open('case_pattern.txt', 'w') as f:
    for i, chunks in enumerate(cabocha_reader(), 1):
        for j, chunk in enumerate(chunks):
            if chunk.is_include_verb() and len(chunk.srcs) > 0:
                joshi = []
                for src in chunk.srcs:
                    predicate = chunk.predicate()
                    if chunks[src].is_include_joshi():
                        joshi.append(chunks[src].joshi())
                if len(joshi) > 0:
                    # print('{}\t{}'.format(predicate, ' '.join(joshi)))
                    line = '{}\t{}\n'.format(predicate, ' '.join(joshi))
                    f.write(line)

In [32]:
!sort case_pattern.txt | uniq -c | sort -r | head -10

 704 云う	と
 452 する	を
 333 思う	と
 202 ある	が
 199 なる	に
 188 する	に
 175 見る	て
 159 する	と
 116 する	が
  98 見る	を
sort: write failed: standard output: Broken pipe
sort: write error


In [33]:
!grep "^する\s" case_pattern.txt | sort | uniq -c | sort -r | head -10

 452 する	を
 188 する	に
 159 する	と
 116 する	が
  88 する	て を
  85 する	は
  61 する	を に
  61 する	て
  60 する	も
  54 する	が を


In [34]:
!grep "^見る\s" case_pattern.txt | sort | uniq -c | sort -r | head -10

 175 見る	て
  98 見る	を
  23 見る	て て
  20 見る	から
  17 見る	と
  13 見る	は て
  12 見る	て を
  12 見る	で
  11 見る	から て
   9 見る	に


In [35]:
!grep "^与える\s" case_pattern.txt | sort | uniq -c | sort -r | head -10

   4 与える	に を
   2 与える	て に を
   1 与える	けれども は を
   1 与える	として か
   1 与える	が は は と て に を
   1 与える	は て に を に
   1 与える	は て に を
   1 与える	と は て を
   1 与える	て は に を
   1 与える	は は も


## 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

* 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
* 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる

「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

```
始める  で      ここで
見る    は を   吾輩は ものを
```

In [36]:
# 最初の10件のみ表示
total = 0
to = 10
for i, chunks in enumerate(cabocha_reader(), 1):
    if total >= to:
        break
    for j, chunk in enumerate(chunks):
        if total >= to:
            break
        if chunk.is_include_verb() and len(chunk.srcs) > 0:
            joshi = []
            src_surfaces = []
            for src in chunk.srcs:
                predicate = chunk.predicate()
                if chunks[src].is_include_joshi():
                    joshi.append(chunks[src].joshi())
                    src_surfaces.append(chunks[src].surface())
            if len(joshi) > 0:
                line = '{}\t{}\t{}'.format(predicate, ' '.join(joshi), ' '.join(src_surfaces))
                print(line)
                total += 1

生れる	で	どこで
つく	か が	生れたか 見当が
泣く	で	所で
する	て は	泣いて いた事だけは
始める	で	ここで
見る	は を	吾輩は ものを
聞く	で	あとで
捕える	を	我々を
煮る	て	捕えて
食う	て	煮て


## 47. 機能動詞構文のマイニング
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

* 「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
* 述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
* 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
* 述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）

例えば「別段くるにも及ばんさと、主人は手紙に返事をする。」という文から，以下の出力が得られるはずである．

```
返事をする      と に は        及ばんさと 手紙に 主人は
```

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

* コーパス中で頻出する述語（サ変接続名詞+を+動詞）
* コーパス中で頻出する述語と助詞パターン

In [37]:
with open('sahen_wo_verb.txt', 'w') as f:
    for i, chunks in enumerate(cabocha_reader(), 1):
        for j, chunk in enumerate(chunks):
            if chunk.is_include_sahen_wo() and chunks[chunk.dst].is_include_verb():
                predicate = chunk.sahen_wo() + chunks[chunk.dst].predicate()

                joshi = []
                src_surfaces = []
                for src in chunk.srcs:
                    if chunks[src].is_include_joshi():
                        joshi.append(chunks[src].joshi())
                        src_surfaces.append(chunks[src].surface())
                for src in chunks[chunk.dst].srcs:
                    if src == j:
                        continue
                    if chunks[src].is_include_joshi():
                        joshi.append(chunks[src].joshi())
                        src_surfaces.append(chunks[src].surface())

                if len(joshi) > 0:
                    line = '{}\t{}\t{}\n'.format(predicate, ' '.join(joshi), ' '.join(src_surfaces))
                    f.write(line)

In [38]:
!cut -f1 sahen_wo_verb.txt | sort | uniq -c | sort -r | head -10

  26 返事をする
  20 挨拶をする
  14 話をする
   8 質問をする
   8 真似をする
   8 喧嘩をする
   5 質問をかける
   5 相談をする
   5 注意をする
   5 昼寝をする


In [39]:
!cut -f1,2 sahen_wo_verb.txt | sort | uniq -c | sort -r | head -10

   5 返事をする	と
   4 挨拶をする	と
   3 質問をかける	と は
   3 挨拶をする	から
   3 返事をする	は と
   3 喧嘩をする	と
   2 同情を表する	と は て
   2 挨拶をする	と も
   2 議論をする	て
   2 真似をする	の


## 48. 名詞から根へのパスの抽出
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

* 各文節は（表層形の）形態素列で表現する
* パスの開始文節から終了文節に至るまで，各文節の表現を"->"で連結する

「吾輩はここで始めて人間というものを見た」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

```
吾輩は -> 見た
ここで -> 始めて -> 人間という -> ものを -> 見た
人間という -> ものを -> 見た
ものを -> 見た
```

In [40]:
def chunk_path(chunks, i):
    path = []
    chunk = chunks[i]
    path.append(chunk)
    
    if chunk.dst != -1:
        path += chunk_path(chunks, chunk.dst)
    return path

In [41]:
# 最初の10件のみ表示
total = 0
to = 10
for i, chunks in enumerate(cabocha_reader(), 1):
    if total >= to:
        break
    for j, chunk in enumerate(chunks):
        if total >= to:
            break
        if chunk.is_include_noun():
            path = chunk_path(chunks, j)
            print(' ->'.join([c.surface() for c in path]))
            total += 1

一
吾輩は ->猫である
猫である
名前は ->無い
どこで ->生れたか ->つかぬ
見当が ->つかぬ
何でも ->薄暗い ->所で ->泣いて ->記憶している
所で ->泣いて ->記憶している
ニャーニャー ->泣いて ->記憶している
いた事だけは ->記憶している


## 49. 名詞間の係り受けパスの抽出
文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がiiとjj（i<ji<j）のとき，係り受けパスは以下の仕様を満たすものとする．

* 問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を"->"で連結して表現する
* 文節iiとjjに含まれる名詞句はそれぞれ，XとYに置換する

また，係り受けパスの形状は，以下の2通りが考えられる

* 文節iiから構文木の根に至る経路上に文節jjが存在する場合: 文節iiから文節jjのパスを表示
* 上記以外で，文節iiと文節jjから構文木の根に至る経路上で共通の文節kkで交わる場合: 文節iiから文節kkに至る直前のパスと文節jjから文節kkに至る直前までのパス，文節kkの内容を"|"で連結して表示

例えば，「吾輩はここで始めて人間というものを見た。」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

```
Xは | Yで -> 始めて -> 人間という -> ものを | 見た
Xは | Yという -> ものを | 見た
Xは | Yを | 見た
Xで -> 始めて -> Y
Xで -> 始めて -> 人間という -> Y
Xという -> Y
```

In [42]:
def merge_path(chunks, chunk_path_X, chunk_path_Y):
    noun_X = chunk_path_X[0]
    noun_Y = chunk_path_Y[0]
    ret = []
    
    # Xで -> 始めて -> Y
    if noun_Y in chunk_path_X:
        idx = chunk_path_X.index(noun_Y)
        for i, c in enumerate(chunk_path_X[:idx]):
            if i == 0:
                ret.append(c.masked_noun_phrase('X'))
            else:
                ret.append(' -> ')
                ret.append(c.surface())
        ret.append(' -> ')
        ret.append(noun_Y.masked_noun('Y'))
            
        return ''.join(ret)
    else: #Xは | Yで -> 始めて -> 人間という -> ものを | 見た
        common_chunks = set(chunk_path_X) & set(chunk_path_Y)
        if len(common_chunks) == 1:
            common_chunk = list(common_chunks)[0]
            idx_X = chunk_path_X.index(common_chunk)
            idx_Y = chunk_path_Y.index(common_chunk)
            for i, c in enumerate(chunk_path_X[0:idx_X]):
                if i == 0:
                    ret.append(c.masked_noun_phrase('X'))
                else:
                    ret.append(' -> ')
                    ret.append(c.surface())
            ret.append(' | ')
            for i, c in enumerate(chunk_path_Y[0:idx_Y]):
                if i == 0:
                    ret.append(c.masked_noun_phrase('Y'))
                else:
                    ret.append(' -> ')
                    ret.append(c.surface())
            ret.append(' | ')
            ret.append(common_chunk.surface())
            return ''.join(ret)
        else:
            return ""

# 最初の10件のみ表示
total = 0
to = 10
for i, chunks in enumerate(cabocha_reader(), 1):
    if total >= to:
        break
    noun_chunks = [c for c in chunks if c.is_include_noun()]
    for j, chunk in enumerate(noun_chunks):
        X_dst = chunk.dst
        X_path = chunk_path(chunks, chunks.index(chunk))
        for k in range(j+1, len(noun_chunks)):
            Y_path = chunk_path(chunks, chunks.index(noun_chunks[k]))
            #print("X:", [c.surface() for c in X_path])
            #print("Y:", [c.surface() for c in Y_path])
            path = merge_path(chunks, X_path, Y_path)
            if len(path) > 0:
                print(path)
                # print("\n")
                total += 1

Xは -> Y
　Xで -> 生れたか | Yが | つかぬ
Xでも -> 薄暗い -> Y
Xでも -> 薄暗い -> 所で -> 泣いて | Yだけは | 記憶している
Xでも -> 薄暗い -> 所で -> 泣いて -> Y
Xで -> 泣いて | Yだけは | 記憶している
Xで -> 泣いて -> Y
X -> 泣いて | Yだけは | 記憶している
X -> 泣いて -> Y
Xだけは -> Y
